In [272]:
from collections import defaultdict
import random
import re
from math import log, exp
import numpy as np

In [273]:
%ls corpora

AnnaKarenina.txt    WarAndPeace.txt     WarAndPeaceEng.txt


In [274]:
with open('corpora/AnnaKarenina.txt') as f:
    txt_1 = f.readlines()

with open('corpora/WarAndPeace.txt') as f:
    txt_2 = f.readlines()

In [275]:
ALPHABET = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя ')

In [276]:
TEST_SIZE = 4
MIN_SENTENCE_TEST_LENGTH = 20

In [277]:
train = []
for line in txt_1 + txt_1:
    train.append(''.join([char for char in line.lower() if char in ALPHABET]))
random.shuffle(train)

In [278]:
test = []
for i, test_sentence in enumerate(train.copy()):
    if len(test_sentence) > MIN_SENTENCE_TEST_LENGTH:
        test.append(test_sentence)
        train.pop(i)
        if len(test) == TEST_SIZE:
            break

In [279]:
test_original = test.copy()
test_code_dict = {orig_char: encoded_char for orig_char, encoded_char in zip(ALPHABET, random.sample(list(ALPHABET), len(ALPHABET)))}
test_coded = [''.join([test_code_dict[char] for char in sentence]) for sentence in test_original]

In [280]:
test_coded

['есернекхсбдтнепесххдфесубсмъчрхдлеепыудпэёеччерчпёдрпбэлеедхседцчхъеыуэрёчбсщчёъхсиезчхюэхс',
 'еиеёсбчиепыусйэрсёсеьдрдуэщепчлцсперчкчщебепчачерердщцэхшеадьсщепщуспщъеьдрдуищексщчтеэернмсёэ',
 'кёдшыдщучаёищъещчёчьусядтеяусхв',
 'ехэбсбехчщеасунхиемдтсемсернепебунёъвсеыдзсёшлщчещстеёфмэечпщъедщдыушщеедщрчцсёепсмдрхэб']

In [281]:
test_original

[' а вы знакомы с анною аркадьевной  спросил ее весловский  она очень привлекательная женщина',
 ' я лакея спрашивала говорит сейчас везет к себе в вотчину богат страсть говорят затем и выдали',
 'злоупотреблять телеграфом франц',
 ' никак нет барыня дома да вы с крыльца пожалуйте там люди есть отопрут  отвечал садовник']

In [282]:
def accuracy(decoded_text):
    acc = 0
    total_length = 0
    for original, decoded in zip(test_original, decoded_text):
        for o_char, d_char in zip(original, decoded):
            if o_char == d_char:
                acc += 1
            total_length += 1
    return acc / total_length

# Базовый частотный метод

In [283]:
def get_frequencies(text):
    frequencies = defaultdict(int)
    for line in text:
        for char in line:
            frequencies[char] += 1
    return frequencies

In [284]:
frequencies_train = get_frequencies(train)
frequencies_test = get_frequencies(test_coded)

In [285]:
sorted_frequencies_train = dict(sorted(frequencies_train.items(), key=lambda x: x[1], reverse=True))
sorted_frequencies_test = dict(sorted(frequencies_test.items(), key=lambda x: x[1], reverse=True))

In [286]:
decode_dict = {list(sorted_frequencies_test.keys())[i]: list(sorted_frequencies_train.keys())[i]
               for i in range(len(sorted_frequencies_test))}

In [287]:
test_decoded = [''.join([decode_dict[char] for char in sentence]) for sentence in test_coded]

In [288]:
print('original:')
print(test_original)

original:
[' а вы знакомы с анною аркадьевной  спросил ее весловский  она очень привлекательная женщина', ' я лакея спрашивала говорит сейчас везет к себе в вотчину богат страсть говорят затем и выдали', 'злоупотреблять телеграфом франц', ' никак нет барыня дома да вы с крыльца пожалуйте там люди есть отопрут  отвечал садовник']


In [289]:
print('decoded:')
print(test_decoded)

decoded:
[' о им ытокеьм с оттей овкоудаитег  спвесрл аа иаслеискрг  ето ебатд пврилаконалдтоя шатэрто', ' я локая спвощриоло чеиеврн сагбос иаыан к саза и иенбртж зечон снвоснд чеиевян ыонаь р имуолр', 'ылежпенвазлянд налачвохеь хвотю', ' тркок тан зовмтя уеьо уо им с квмлдюо пешолжгна ноь лйур аснд енепвжн  ениабол соуеитрк']


In [290]:
print(f'char accuracy: {accuracy(test_decoded):.2f}')

char accuracy: 0.33


# Биграммный частотный метод

In [291]:
def get_ngramm_frequencies(text, ngram):
    frequencies = defaultdict(int)
    for line in text:
        for idx in range(len(line) - ngram + 1):
            frequencies[line[idx:idx + ngram]] += 1
    return frequencies

In [292]:
def decode_ngramm(text, decoder, ngram):
    decoded = text.copy()
    i = 0
    for i in range(len(decoded)):
        line = decoded[i]
        for key in decoder:
            if len(line) < ngram:
                break
            for match in re.finditer(key, line):
                decoded[i] = decoded[i][:match.start()] + decoder[key] + decoded[i][match.end():]
                line = line[:match.start()] + line[match.end():]
    return decoded

In [293]:
frequencies_train = get_ngramm_frequencies(train, 2)
frequencies_test = get_ngramm_frequencies(test_coded, 2)

In [294]:
sorted_frequencies_train = dict(sorted(frequencies_train.items(), key=lambda x: x[1], reverse=True))
sorted_frequencies_test = dict(sorted(frequencies_test.items(), key=lambda x: x[1], reverse=True))

In [295]:
decode_dict = {list(sorted_frequencies_test.keys())[i]: list(sorted_frequencies_train.keys())[i]
               for i in range(len(sorted_frequencies_test))}

In [296]:
test_decoded = decode_ngramm(test_coded, decode_dict, 2)

In [297]:
print('original:')
print(test_original)

original:
[' а вы знакомы с анною аркадьевной  спросил ее весловский  она очень привлекательная женщина', ' я лакея спрашивала говорит сейчас везет к себе в вотчину богат страсть говорят затем и выдали', 'злоупотреблять телеграфом франц', ' никак нет барыня дома да вы с крыльца пожалуйте там люди есть отопрут  отвечал садовник']


In [298]:
print('decoded:')
print(test_decoded)

decoded:
['но кн зеср  аннотоддор м ммъчворлпр ыудп в ччточпё па эя одхседцчхъеыу нёчбсщчё нсиезчхюэхс', 'кимлл скем хсом в к ьдоруэли чеи пенлачщг де ачереи вцне оадьсщои успщъеьдрдуище сщчтеэернмсёэ', 'крушдиеуч мищ о качьусянаеяусхв', 'иложе хеав ас ваиарлом рие  ч я  е лаоеыдзсёкощчещсттофм я пщи дщды вщеедщрчцсёо смдрхэб']


In [299]:
print(f'char accuracy: {accuracy(test_decoded):.2f}')

char accuracy: 0.04


# MCMC

Текст может быть представлен в виде марквской цепи, т.к. след. символ зависит от предыдущего.

MCMC Метрополиса-Гастингса с использованием статистики биграмм будет выглядеть след. образом:

1. Случайным образом меняем словарь для декодирования(будем менять 2 символа).
2. Декодируем текст используя словарь
3. Находим все биграммы в декодированном тексте и считаем loglikelihood найденных биграмм, используя распределение биграмм на train корпусе
4. Если полученное правдоподобие больше, принимаем новый словарь. В противном случае, принимаем его, с вероятностью $\exp(\text{loglikelihood}_{\text{new}} - \text{loglikelihood}_{\text{old}})$
5. Сохраняем словарь с наибольшим показателем loglikelihood

In [340]:
class MCMC:
    def __init__(self, train, ngramms):
        self.ngramms = ngramms
        self._init_train_ngramms_distribution(train)
        self.min_frequency = min(self.train_ngramms_distribution.values()) / 2
        self.sorted_train_onegramm = [item[0] for item in sorted(get_frequencies(train).items(), key=lambda x: x[1], reverse=True)]

    def fit(self, test, iterations):
        sorted_train_onegramm = self.sorted_train_onegramm.copy()
        sorted_test_onegramm = [item[0] for item in sorted(get_frequencies(test).items(), key=lambda x: x[1], reverse=True)]
        self.decode_dict = self._make_decoder(sorted_train_onegramm, sorted_test_onegramm)
        likelyhood = self._ngram_log_likelyhood(test, self.decode_dict)
        best_likelyhood = likelyhood
        for i in range(iterations):
            new_sorted_train_onegramm = self._q_sampling(sorted_train_onegramm)
            new_decode_dict = self._make_decoder(new_sorted_train_onegramm, sorted_test_onegramm)
            new_likelyhood = self._ngram_log_likelyhood(test, new_decode_dict)
            if self.metropolis_hastings_log_accept(likelyhood, new_likelyhood):
                likelyhood = new_likelyhood
                sorted_train_onegramm = new_sorted_train_onegramm
                if new_likelyhood > best_likelyhood:
                    best_likelyhood = new_likelyhood
                    self.decode_dict = new_decode_dict
        return likelyhood

    def predict(self, test):
        return [''.join([self.decode_dict[char] for char in sentence]) for sentence in test]

    @staticmethod
    def metropolis_hastings_log_accept(l, l_new):
        if l_new > l:
            return True
        else:
            return random.random() < (exp(l_new - l))
            
    @staticmethod
    def _q_sampling(sorted_train_onegramm):
        ind1, ind2 = random.sample(range(len(sorted_train_onegramm)), k=2)
        sorted_train_onegramm_copy = sorted_train_onegramm.copy()
        sorted_train_onegramm_copy[ind1], sorted_train_onegramm_copy[ind2] = sorted_train_onegramm_copy[ind2], sorted_train_onegramm_copy[ind1]
        return sorted_train_onegramm_copy

    def _init_train_ngramms_distribution(self, train):
        self.train_ngramms_distribution = get_ngramm_frequencies(train, self.ngramms)
        factor=1.0 / sum(self.train_ngramms_distribution.values())
        for k in self.train_ngramms_distribution:
            self.train_ngramms_distribution[k] = self.train_ngramms_distribution[k] * factor

    @staticmethod
    def _make_decoder(sorted_train_onegramm, sorted_test_onegramm):
        return {key: val for key, val in zip(sorted_test_onegramm, sorted_train_onegramm)}

    def _ngram_log_likelyhood(self, text, decoder):
        log_likelyhood = 0
        for line in text:
            line_decoded = [decoder[char] for char in line]
            for idx in range(len(line_decoded) - self.ngramms + 1):
                log_likelyhood += log(self.train_ngramms_distribution.get(''.join(line_decoded[idx:idx + self.ngramms]), self.min_frequency))
        return log_likelyhood

In [341]:
mcmc = MCMC(train, 2)
mcmc.fit(test_coded, 15000)

-1687.1493687208983

In [342]:
test_decoded = mcmc.predict(test_coded)

In [343]:
print('original:')
print(test_original)

original:
[' а вы знакомы с анною аркадьевной  спросил ее весловский  она очень привлекательная женщина', ' я лакея спрашивала говорит сейчас везет к себе в вотчину богат страсть говорят затем и выдали', 'злоупотреблять телеграфом франц', ' никак нет барыня дома да вы с крыльца пожалуйте там люди есть отопрут  отвечал садовник']


In [344]:
print('decoded:')
print(test_decoded)

decoded:
[' а вы знакомы с анною аркадьевной  спросил ее весловский  она очень привлекательная женщина', ' я лакея спрашивала говорит сейчас везет к себе в вотчину богат страсть говорят затем и выдали', 'злоупотреблять телеграхом хранц', ' никак нет барыня дома да вы с крыльца пожалуйте там люди есть отопрут  отвечал садовник']


In [345]:
print(f'char accuracy: {accuracy(test_decoded):.4f}')

char accuracy: 0.9934


# Тестовый пример

In [367]:
test = ["←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"]
test

['←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏']

In [368]:
mcmc.fit(test, 100000)

-1240.4133698882958

In [369]:
mcmc.predict(test)

['если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж']

# Бонус многограммы

In [334]:
n_gramms = [2, 3, 4, 5]
attempts = 10
for n_gramm in n_gramms:
    likelyhood_max = -float('inf')
    mcmc = MCMC(train, n_gramm)
    best_decode_dict = None
    for _ in range(attempts):
        likelyhood = mcmc.fit(test, 20000)
        if likelyhood > likelyhood_max:
            best_decode_dict = mcmc.decode_dict
            likelyhood_max = likelyhood
    mcmc.decode_dict = best_decode_dict
    predict = mcmc.predict(test)
    print(f'n_gramm: {n_gramm}')
    print('predict')
    print(predict)

n_gramm: 2
predict
['если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж']
n_gramm: 3
predict
['если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю']
n_gramm: 4
predict
['если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю']
n_gramm: 5
predict
['если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я нич

с увеличением n_gram растет кол-во локальных максимумов и приходится много раз перезапускать алгоритм, но заметно, что n_gram=3, достаточно для идеального декодирования тестового текста

# Бонус применение

Скорее всего можно применять как baseline перевода, для языков о которых мы ничего не знаем(к примеру с животного на английский)